In [1]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config")
os.environ.setdefault("DJANGO_CONFIGURATION", "Local")

import configurations
configurations.setup()

# import django
from django.conf import settings
# settings.configure(DEBUG=True)
# django.setup()


In [2]:
# Now this script or any imported module can use any part of Django it needs.
from djstruct.models import DjangoBaseNode, DjangoDependencyRelation

In [3]:
from yamlstore.loaders import read_data_dir
from yamlstore.loaders import PrerequisteReference, UsedforReference, RelatedReference
from yamlstore.loaders import NodeFromYamlDict

IMPORT_DATA_DIR = '/Users/ivan/Projects/Minireference/STRUCTURE/structure-api/data/miniref'

all_data = read_data_dir(IMPORT_DATA_DIR)


In [4]:
len(all_data)

22

In [5]:
all_data[3]

{'__class__': 'Subject',
 'books': "Any linear algebra book. Anton's is decent.",
 'description': 'Linear algebra deals with vectors, linear transformations, matrices, linear equations, vector spaces, projections, and many other topics. This is one of the most important subjects for undergraduates to learn well because linear algebra techniques are used all over the place. The computational techniques, geometric intuition, and theoretical analysis all combine to form a mathematical modelling toolbox that students can then use to tackle problems in mathematics, physics, chemistry, engineering, computing, business, economics, and even the social sciences. Linear algebra is also an important stepping stone towards graduate-level mathematics.\n',
 'level': 'UGRAD',
 'path': 'math/linear_algebra',
 'prerequisites': ['math/numbers',
  'math/functions',
  'math/algebra',
  'math/geometry'],
 'relations': None,
 'scope': 'miniref',
 'usedfors': [{'explain': 'The first linear algebra course is 

In [6]:
from yamlstore.loaders import dicts_to_objects, extact_paths

data_objects = dicts_to_objects(all_data)

all_paths, all_path_refs = extact_paths(data_objects)

    

In [7]:
# PASS 0. delete all nodes
for oldnode in DjangoBaseNode.objects.all():
    oldnode.delete()
for oldreln in DjangoDependencyRelation.objects.all():
    oldreln.delete()



In [9]:
from yamlstore.loaders import create_nodes_and_relations


create_nodes_and_relations(data_objects)

In [11]:
print( len( DjangoBaseNode.objects.all() ) )

print( len( DjangoDependencyRelation.objects.all() ) )
# all_paths

22
32


In [ ]:
data_objects[2].__dict__

In [ ]:
#### UNRELATED PROLOG STUFF

perl_funciton_name = 'prerequisites_for'

def slashes_to_underscores(subj):
    return subj.replace('/','_')

def knowify(subj):
    return 'K_'+slashes_to_underscores(subj)

def learnify(subj):
    return 'L_'+slashes_to_underscores(subj)

def make_prolog_statement(subject):
    prereqs_list = []
    if 'prerequisites' in subject and subject['prerequisites'] is not None:
        for prereq_str_or_dict in subject['prerequisites']:
            prereq = PrerequisteReference(prereq_str_or_dict)
            prereqs_list.append(prereq.prerequisite)
    prolog_statement = ''
    prolog_statement += perl_funciton_name + '('
    prolog_statement += '['
    prolog_statement += ','.join( map(knowify, prereqs_list) )
    prolog_statement += '],'
    prolog_statement += learnify(subject['path'])
    prolog_statement += ').'
    return prolog_statement

for subject in all_data:
    print make_prolog_statement(subject)


In [ ]:
l  = [1,2,3]

In [ ]:
l.extend( [3,4])

In [ ]:
l

In [ ]:
d = {'a':3, 'b':4}

In [ ]:
d.update( {'c':55} )

In [ ]:
d

In [ ]:
d.update({'a':6})

In [ ]:
d